In [1]:
from collections import namedtuple
# neptune
import os
import numpy as np

from time import sleep
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.nn import functional as F
from models.t5 import *
from data.data import *
device="cuda"

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gersa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
model = T5FinetuneForRACE.load_from_checkpoint("D:/Github/decepticon/project/checkpoints/0_02.ckpt").to(device)

In [3]:
class Args:
    def __init__(self):
        self.data_path =  "D:\Github/decepticon/Processed_New"
        self.batch_size =  16
        self.num_workers =  0
        self.pretrained_model = "t5-small"
        self.tokenizer_len = 32102 ## manual
        self.padding_id = 0
    def to_dict(self):
        out = dict()
        out["data_path"] = self.data_path
        out["batch_size"] = self.batch_size
        out["num_workers"] = self.num_workers
        out["pretrained_model"] = self.pretrained_model
        out["tokenizer_len"] = self.tokenizer_len
        out["padding_id"] = self.padding_id
        return out

hparams=Args()

In [4]:
data_module = RaceDataModule(hparams)
data_module.prepare_data()
data_module.setup()

SETUP: Training Dataset
SETUP: Validation Dataset
SETUP: Test Dataset


In [9]:
def generate_question(content):
    model.eval()        
    generated = model.model.generate(input_ids = x["input_ids"].to(device), 
                                     num_beams = 6,
                                     num_return_sequences = 5,
                                     max_length = 50,
                                     no_repeat_ngram_size = 2,
                                     early_stopping = True)

    output = []
    for sequence in generated[:5]:
        output.append(data_module.tokenizer.decode(sequence, skip_special_tokens= True, clean_up_tokenization_spaces = True))
    content = data_module.tokenizer.decode(content["input_ids"][0], skip_special_tokens= False, clean_up_tokenization_spaces = True)
    return output, content

In [10]:
j = 0
for x, y in data_module.val_dataloader():
    output = generate_question(x)
    print("-----------------------------------------")
    print("GROUND TRUTH: %s" %data_module.tokenizer.decode(y["input_ids"][0], skip_special_tokens= True, clean_up_tokenization_spaces = True))
    for i, q in enumerate(output[0]):
        print("GENERATED %s: %s"  %(i, "".join(q)))
    print("ANSWER: %s" % "".join(output[1]).split("<context>")[0].replace("<answer>", "").lstrip())
    j += 1
    if j > 10: break

-----------------------------------------
GROUND TRUTH: The Sherman Antitrust Act _.
GENERATED 0: The Sherman Antitrust Act _.
GENERATED 1: According to the passage, the early tycoons _.
GENERATED 2: It can be inferred from the passage that the Sherman Antitrust Act _.
GENERATED 3: The first tycoons built successful businesses because they _.
GENERATED 4: The Sherman Antitrust Act was passed because it _.
ANSWER: sought to eliminate monopolies in favor of competition in the market-place
-----------------------------------------
GROUND TRUTH: Why are ECAs important now?
GENERATED 0: What does the passage mainly talk about?
GENERATED 1: Which of the following is TRUE about ECAs?
GENERATED 2: What does the passage mainly tell us about ECAs?
GENERATED 3: Which of the following is NOT true about ECAs?
GENERATED 4: What's the main idea of the passage?
ANSWER: They prepare the students for their future.
-----------------------------------------
GROUND TRUTH: Which of the following should be a